# Scanning the BP dataset with pysegy

This notebook demonstrates how to download the BP 1994 2-D seismic dataset, scan it using the `pysegy` utilities and then visualise the source and receiver positions as well as a few shot gathers.

In [ ]:
import sys
sys.path.append('..')  # allow importing the local pysegy package
import pysegy
import urllib.request, gzip, os, shutil
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
path = '../data/overthrust_2D_shot_1_20.segy'

In [ ]:
scan = pysegy.segy_scan(path)
fh = scan.fileheader
print('Total shots:', len(scan))
print('Samples per trace:', fh.bfh.ns)

In [ ]:
# Display human-readable headers
fh

In [ ]:
example_hdr = scan.read_headers(0, keys=['SourceX', 'GroupX', 'ns', "dt", "SourceDepth"])[0]
example_hdr

In [ ]:
# Access ShotRecord directly and use lazy data
rec = scan[0]
hdr = rec.read_headers(keys=['SourceX'])[0]
block = rec.data
block

In [ ]:
# Receiver coordinates for a shot
rec = scan[0]
coords = rec.rec_coordinates
plt.figure(figsize=(6,4))
plt.scatter(coords[:,0], coords[:,1], s=2)
plt.xlabel('GroupX')
plt.ylabel('GroupY')
plt.title('Receiver positions for first shot')
plt.show()

In [ ]:

# Plot a few shot gathers using the lazy reader
for i in range(3):
    block = scan[i]
    print(f'Shot {i+1} - Samples: {block.data.shape[0]}, Traces: {block.data.shape[1]}')
    plt.figure(figsize=(8,4))
    plt.imshow(block.data, aspect='auto', cmap='gray', vmin=-.0001, vmax=.0001)
    plt.title(f'Shot gather {i+1}')
    plt.xlabel('Trace')
    plt.ylabel('Sample')
    plt.show()


In [ ]:

# Receiver spread across all sources
xvals = []
src_ids = []
for i in range(len(scan)):
    hdrs = scan.read_headers(i, keys=['GroupX'])
    xvals.extend(h.GroupX for h in hdrs)
    src_ids.extend([i] * len(hdrs))
plt.figure(figsize=(8,5))
plt.scatter(xvals, src_ids, s=2)
plt.xlabel('X')
plt.ylabel('Source number')
plt.title('Receiver spread')
plt.show()


In [ ]:
assert len(scan.shots) == len(scan.counts)
assert scan.fileheader.bfh.ns == scan[0].fileheader.bfh.ns